<h1 align= "center"><b>Problem: Running With Bunnies</b></h1>

You and your rescued bunny prisoners need to get out of this collapsing death trap of a space station - and fast! Unfortunately, some of the bunnies have been weakened by their long imprisonment and can't run very fast. Their friends are trying to help them, but this escape would go a lot faster if you also pitched in. The defensive bulkhead doors have begun to close, and if you don't make it through in time, you'll be trapped! You need to grab as many bunnies as you can and get through the bulkheads before they close. 

The time it takes to move from your starting point to all of the bunnies and to the bulkhead will be given to you in a square matrix of integers. Each row will tell you the time it takes to get to the start, first bunny, second bunny, ..., last bunny, and the bulkhead in that order. The order of the rows follows the same pattern (start, each bunny, bulkhead). The bunnies can jump into your arms, so picking them up is instantaneous, and arriving at the bulkhead at the same time as it seals still allows for a successful, if dramatic, escape. (Don't worry, any bunnies you don't pick up will be able to escape with you since they no longer have to carry the ones you did pick up.) You can revisit different spots if you wish, and moving to the bulkhead doesn't mean you have to immediately leave - you can move to and from the bulkhead to pick up additional bunnies if time permits.

In addition to spending time traveling between bunnies, some paths interact with the space station's security checkpoints and add time back to the clock. Adding time to the clock will delay the closing of the bulkhead doors, and if the time goes back up to 0 or a positive number after the doors have already closed, it triggers the bulkhead to reopen. Therefore, it might be possible to walk in a circle and keep gaining time: that is, each time a path is traversed, the same amount of time is used or added.

Write a function of the form solution(times, time_limit) to calculate the most bunnies you can pick up and which bunnies they are, while still escaping through the bulkhead before the doors close for good. If there are multiple sets of bunnies of the same size, return the set of bunnies with the lowest prisoner IDs (as indexes) in sorted order. The bunnies are represented as a sorted list by prisoner ID, with the first bunny being 0. There are at most 5 bunnies, and time_limit is a non-negative integer that is at most 999.

For instance, in the case of

    [
        [0, 2, 2, 2, -1],  # 0 = Start
        [9, 0, 2, 2, -1],  # 1 = Bunny 0
        [9, 3, 0, 2, -1],  # 2 = Bunny 1
        [9, 3, 2, 0, -1],  # 3 = Bunny 2
        [9, 3, 2, 2,  0],  # 4 = Bulkhead
    ]

and a time limit of 1, the five inner array rows designate the starting point, bunny 0, bunny 1, bunny 2, and the bulkhead door exit respectively. You could take the path:

Start End Delta Time Status:

    -   0     -    1 Bulkhead initially open
    0   4    -1    2
    4   2     2    0
    2   4    -1    1
    4   3     2   -1 Bulkhead closes
    3   4    -1    0 Bulkhead reopens; you and the bunnies exit

With this solution, you would pick up bunnies 1 and 2. This is the best combination for this space station hallway, so the answer is [1, 2].

<h3 align= "center"><b>Test Cases</b></h3>

```

Input:
    solution.solution([[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 3)
Output:
    [0, 1]

Input:
    solution.solution([[0, 2, 2, 2, -1], [9, 0, 2, 2, -1], [9, 3, 0, 2, -1], [9, 3, 2, 0, -1], [9, 3, 2, 2, 0]], 1)
Output:
    [1, 2]

```


<h3 align= "center"><b>The idea</b></h3>

- First idea: if there is a cycle that increases time limit then you can always go there first, increase the time limit as much as needed to pick up all bunnies and exit with all the bunnies. So we can check that first.

- Second idea: We can traverse all the possibilities for picking bunnies with a BFS, keeping track of the best possible time to get to the previously visited positions carrying each different possible combination of bunnies. We only visit a position again if we improve that time or carry different bunnies. Since there isn't any cycle that increases the time limit, then this will eventually end (proof left as exercise :P) and then we can just choose the best possible answer that made it to the exit in time.


<h3 align= "center"><b>The code</b></h3>

In [1]:
def solution(times, times_limit):
    
    size = len(times)
    
    # check for a cycle that increase time limit, can't be longer than number of nodes or would contain a smaller one
    # note you can always decide to stay so this checks for cycles shorter than number of nodes too
    for start in range(size):
        time_to = [ row[start] for row in times ]
        for step in range(size):
            new_time_to = [
                min ( [ time_to[passing] + times[new_to][passing] for passing in range(size)] )
                for new_to in range(size)
            ]
            time_to = new_time_to
        
        if time_to[start] < 0:
            # if there's such a cycle you can increase time limit as much as needed to pick up all bunnies
            return range(size - 2)

    # BFS keeping track of current position, bunnies already picked up and time to that position
    starting = [False for _ in range(size)]
    starting[0] = True
    best_times = { (0, tuple(starting)) : 0 }
    queue = [ ((0, tuple(starting)) , 0) ]
    ans = []
    while queue:
        ((at,holding) , time) = queue.pop()
        
        if at == size-1 and time <= times_limit:
            holding_as_list = [i-1 for i in range(1,size-1) if holding[i]]
            if len(holding_as_list) > len(ans) or (len(holding_as_list) == len(ans) and holding_as_list < ans):
                ans = holding_as_list
        
        for new_at, time_to_new_at in enumerate(times[at]):
            holding_new_at = list(holding)
            holding_new_at[new_at] = True
            new_status = (new_at, tuple(holding_new_at))
            new_time = time + time_to_new_at
            if new_status not in best_times or new_time < best_times[new_status]:
                best_times[new_status] = new_time
                queue.append((new_status, new_time))
    
    return ans


In [2]:
print (solution([[0, 2, 2, 2, -1], [9, 0, 2, 2, -1], [9, 3, 0, 2, -1], [9, 3, 2, 0, -1], [9, 3, 2, 2, 0]], 1))

[1, 2]


In [3]:
print(solution([[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 3))

[0, 1]
